In [1]:
import torch
import torch.nn as nn
import logging

from comblearn.data import DSFValueFunction
from comblearn.env import CombinatorialAuction

logging.basicConfig(level=20)

In [2]:
import yaml 

with open("../config/config_mod.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)

In [3]:
device = "cuda"

def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [4]:
auction = CombinatorialAuction(cfg['auction'])

INFO:root:tensor([0., 0., 6., 1., 1., 1., 3., 4.], grad_fn=<SqueezeBackward0>)


INFO:root:tensor([2., 2., 5., 3., 4., 4., 4., 4.], grad_fn=<SqueezeBackward0>)
INFO:root:tensor([3., 1., 3., 5., 0., 0., 1., 1.], grad_fn=<SqueezeBackward0>)
INFO:root:tensor([4., 0., 1., 6., 4., 0., 2., 4.], grad_fn=<SqueezeBackward0>)
INFO:root:tensor([5., 4., 3., 3., 5., 6., 2., 6.], grad_fn=<SqueezeBackward0>)


In [5]:
allocations, payments = auction.run()

INFO:root:Final allocation calculation...
INFO:root:Bidder ali, loss: 0.004074868746101856
INFO:root:Bidder hassan, loss: 6.783932443577001e-11
INFO:root:Bidder mahdi, loss: 6.615036545554176e-05
INFO:root:Bidder jafar, loss: 6.919712177477777e-05
INFO:root:Bidder arash, loss: 1.800654070294172e-10
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:Optimal allocation:
INFO:root:(('ali', tensor([0., 0., 1., 0., 0., 0., 0., 1.], device='cuda:0')))
INFO:root:(('hassan', tensor([0., 1., 0., 0., 0., 1., 1., 0.], device='cuda:0')))
INFO:root:(('mahdi', tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')))
INFO:root:(('jafar', tensor([0., 0., 0., 1., 0., 0., 0., 0.], device='cuda:0')))
INFO:root:(('arash', tensor([1., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0')))
INFO:root:Social welfare: 35.9562873840332
INFO:root:Payment calculation..
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.500000